In [1]:
import os
import ast
import json
import nltk
import time
import spacy
import openai
import random
import jellyfish
import numpy as np
import pandas as pd
from umap import UMAP
import seaborn as sns
from datetime import date,timedelta
from bertopic import BERTopic
from nltk.tokenize import word_tokenize
from sklearn.metrics import silhouette_score
from dateutil.relativedelta import relativedelta
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import matplotlib.pyplot as plt
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

C:\Users\kshit\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kshit\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\kshit\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def layer_creation(sheet5,layer,multi_product,product=None):
    for senti in ['Positive','Negative','Neutral']:
        if multi_product:
            a = sheet5[(sheet5.Sentiment==senti)&(sheet5.Product==product)]
        else:
            a = sheet5[sheet5.Sentiment==senti]
        L2 = {}
        if layer=='L2':
            x = 'L3'
        else:
            x = 'L2'
        #Calculating the layer values (count,avg) using its next layer vales    
        for aspect in a[layer+' Cluster'].unique():
            b = a[a[layer+' Cluster']==aspect]
            d = {}
            for cluster in b[x+' Cluster'].unique():
                c = b[b[x+' Cluster']==cluster]
                pcount = list(b[b[x+' Cluster']==cluster][x+' Phrase Count'])[0]
                rcnt = list(b[b[x+' Cluster']==cluster][x+' Review Count'])[0]
                prating = list(b[b[x+' Cluster']==cluster][x+' Rating (Phrase)'])[0]
                rrat = list(b[b[x+' Cluster']==cluster][x+' Rating (Review)'])[0]
                d[cluster] = [pcount,prating,rcnt,rrat]
            p_total = 0
            p_product = 0
            r_total = 0
            r_product = 0
            for key,value in d.items():
                try:
                    p_total+=value[0]
                    r_total+=value[2]
                except:
                    continue
                p_product+=(value[0]*value[1])
                r_product+=(value[2]*value[3])
            try:
                L2[aspect] = [p_total,p_product/p_total,r_total,r_product/r_total]
            except:
                continue
        #Inserting them into sheet5
        for aspect,value in L2.items():
            if multi_product:
                for i in sheet5[(sheet5.Product==product) & (sheet5.Sentiment==senti) & (sheet5[layer+' Cluster']==aspect)].index:
                    sheet5[layer+' Phrase Count'][i] = value[0]
                    sheet5[layer+' Rating (Phrase)'][i] = value[1]
                    sheet5[layer+' Review Count'][i] = value[2]
                    sheet5[layer+' Rating (Review)'][i] = value[3]
            else:
                for i in sheet5[(sheet5.Sentiment==senti) & (sheet5[layer+' Cluster']==aspect)].index:
                    sheet5[layer+' Phrase Count'][i] = value[0]
                    sheet5[layer+' Rating (Phrase)'][i] = value[1]
                    sheet5[layer+' Review Count'][i] = value[2]
                    sheet5[layer+' Rating (Review)'][i] = value[3]
    print(layer+" Completed\n")
    return sheet5

In [3]:
def summary_l3(df1,multi_product=False):
    #This code will generate the summary of L3 Clusters and store it in sheet5
    if multi_product:
        l = ['Product','Cluster','Sentiment']
    else:
        l = ['Cluster','Sentiment']
    x = pd.DataFrame(df1.groupby(l).count()['Phrase'])
    x.columns = ['L3 Phrase Count']
    z = pd.DataFrame(df1.groupby(l).mean()['Rating'])
    z.columns = ['L3 Rating (Phrase)']
    a = pd.DataFrame(df1.groupby(l+['Review']).mean().groupby(l).mean()['Rating'])
    a.columns = ['L3 Rating (Review)']
    layer3 = x.join(z).join(a)
    layer3 = layer3.reset_index()
    layer3['L3 Review Count']=''
    if multi_product:
        for i in layer3.index:
            layer3['L3 Review Count'][i] = df1[(df1.Product==layer3['Product'][i])&(df1.Cluster==layer3['Cluster'][i])&(df1.Sentiment==layer3['Sentiment'][i])]['Review ID'].nunique()
    else:
        for i in layer3.index:
            layer3['L3 Review Count'][i] = df1[(df1.Cluster==layer3['Cluster'][i])&(df1.Sentiment==layer3['Sentiment'][i])]['Review ID'].nunique()
    return layer3

In [4]:
def id_creation(sheet5,layer,client):
    if layer==1:
        id = 1
        for u_cluster in sheet5['L1 Cluster'].unique():
            for i in sheet5[sheet5['L1 Cluster']==u_cluster].index:
                sheet5['L1 Cluster ID'][i] = client+'L1I'+ str(id)
            id+=1
    elif layer==2 or layer==3:
        a = 'L'+str(layer-1)+' Cluster ID'
        b = 'L'+str(layer)+' Cluster'
        id = 1
        for ul1_id in sheet5[a].unique():
            for ul2_cluster in sheet5[sheet5[a]==ul1_id][b].unique():
                for i in sheet5[(sheet5[a]==ul1_id) & (sheet5[b]==ul2_cluster)].index:
                    sheet5[b+' ID'][i] = client+'L'+str(layer)+'I'+ str(id)
                id+=1
    else:
        id = 1
        for i in sheet5.index:
            sheet5['L4 ID'][i] = client+'L4I'+ str(id)
            id+=1
    return sheet5

In [5]:
def clustering(df1,n_neighbors,words_per_topic,ng_range,compress,nlp,tokenization,remove_stopwords,aspect):
    docs = list(df1.loc[:, "Phrase"].values)
    
    # Lemmatization
    for i in range(len(docs)):
        doc = nlp(docs[i])
        tokens = []
        for token in doc:
            tokens.append(token)
        lemmatized_sentence = " ".join([token.lemma_ for token in doc])
        docs[i] = lemmatized_sentence

    docs = pd.Series(docs).apply(lambda x: tokenization(x)) #Tokenization is done to remove stopwords
    docs = docs.apply(lambda x:remove_stopwords(x))
    for i in range(len(docs)):
        docs[i] = " ".join(docs[i])

    #This code is used to only retain adverb, verb, adjective, noun, pronoun & 'X' part of speech words
    #For more information about part of speech refer spacy website
    lst = ['ADV','VERB','ADJ','NOUN','PROPN','X','NUM']

    for i in range(len(docs)):
        sent = nlp(docs[i])
        text= []
        for word in sent:
            if not word.is_punct and word.pos_ in lst:#not word.is_stop and
                text.append(word.lemma_) #ps.stem(word.lemma_)

        #This code is used to convert some word into another. This is useful in Topic Modelling
#         suffix = {'hydrating':'hydrate','hydrated':'hydrate','hydration':'hydrate'}
#         for j in range(len(text)):
#             for su,w in suffix.items():
#                 if text[j].endswith(su):
#                     text[j] = text[j][:-len(su)] + w

        #This code is used to remove duplicate words from phrases
        k = []
        st = ' '.join(text)
        for x in text:
            if (st.count(x)>=1 and (x not in k)):
                k.append(x)
#                 k.sort()
        docs[i] = ' '.join(k)

    docs = pd.Series(docs)

    #Removing empty phrases from docs [Those phrases which contains stopwords]
    df1['cp'] = docs
    df1 = df1[df1.cp.astype(bool)]
    docs = docs[docs.astype(bool)]
    df1.index=range(len(df1))
    docs.index=range(len(docs))

    #To enable compression while Topic Modelling
    if compress:
        compress="auto"
    else:
        compress=None

    try:
        modal = BERTopic(
#                      embedding_model = nlp,
                     top_n_words = words_per_topic,
                     calculate_probabilities = True,  #Used in Outlier Reduction
                     vectorizer_model = TfidfVectorizer(  #This vectorizer takes the importance of a word or ngram into consideration using tfidf
#                                              vocabulary = v.get_feature_names_out(),
                                     ngram_range = ng_range,
                                     stop_words="english"
                     ),
                     nr_topics=compress,
                     diversity=1, #To diversify the generated topics. Set to maximum
                     umap_model=UMAP(n_neighbors=n_neighbors,random_state=2), #Dimensionality Reduction Model, random_state is defined to make BERTopic reproducible
                     ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True,bm25_weighting=True), #Used for topic compression
                     language="english",
                     n_gram_range=ng_range) 
        topics, probs = modal.fit_transform(docs.values)

        try:
            #Reducing Outliers [Removing the -1 topic and Making Cleaner Topics as well]
            topics = modal.reduce_outliers(docs.values, topics, probabilities=probs, strategy="probabilities")
            modal.update_topics(docs.values, topics=topics)
        except:
            pass

        #Storing clusters corresponding to each phrase in df1
        df1['Cluster']=''
#                     predicted_topics, predicted_probs = modal.transform(docs)
        #This code is used to remove duplicate words inside a topic name
        for i in range(len(df1)):
            s = modal.get_topic(topics[i])[0][0]
            l = s.split()
            k = []
            for x in l:
                if (s.count(x)>=1 and (x not in k)):
                    k.append(x)
            df1['Cluster'][i] = ' '.join(k)
            
        df1['Cluster'] = df1['Cluster'].str.capitalize()
        print(modal.get_topic_info())
        print(pd.DataFrame(df1['Cluster'].value_counts()))
    except Exception as e:
        print('Aspect contains insufficient number of phrases for clustering. BERTopic Error.\n')
        print(e)
        df1['Cluster']=aspect
        
    return df1

In [6]:
def most_similar_phrase(df1):
    df1['L3 Cluster Phrase']=''
    df1['sim'] = ''
    df1.drop(['index'],1,inplace=True)
#Calculating the mean similarity score corresponding to each phrase & then taking the phrase with max mean similarity score
    for cluster in df1['Cluster'].unique():
        for senti in df1[df1.Cluster==cluster]['Sentiment'].unique():
            dfx = df1[(df1.Cluster==cluster)&(df1.Sentiment==senti)]
            if len(dfx)==1:
                for j in dfx.index:
                    df1['L3 Cluster Phrase'][j] = dfx['Phrase'].iloc[0].strip().capitalize()                   
                continue

            for i in dfx.index:
                sim=[]
                for j in dfx.index:
                    if i!=j:
                        sim.append(jellyfish.levenshtein_distance(df1['Phrase'][i],df1['Phrase'][j]))
                df1['sim'][i] = sum(sim)/len(sim)

            m = df1[(df1.Cluster==cluster)&(df1.Sentiment==senti)]['sim'].min()
            p = list(df1[(df1.Cluster==cluster)&(df1.Sentiment==senti)&(df1['sim']==m)]['Phrase'])[0]
            for i in dfx.index:
                df1['L3 Cluster Phrase'][i] = p.strip().capitalize()                   
    print(df1.loc[:,['Cluster','Sentiment','L3 Cluster Phrase']].sort_values(['Cluster','Sentiment'],ascending=False).drop_duplicates(),'\n')
    return df1

In [7]:
def cluster_merge(l4,multi_product,min_l3_clusters=3,min_l3_phrases=10,percentage=10):
    df = pd.DataFrame(columns = list(l4.columns)+['sim'])
    if multi_product:
        ps = l4['Product'].unique()
    else:
        ps = [1]
    for p in ps:
        if multi_product:
            df1 = l4[l4['Product']==p]
            l2s = df1['L2 Cluster ID'].unique()
        else:
            df1 = l4.copy()
            l2s = df1['L2 Cluster ID'].unique()   
        for l2 in l2s:
            for senti in df1[df1['L2 Cluster ID']==l2]['Sentiment'].unique():
                df2 = df1[(df1['L2 Cluster ID']==l2)&(df1['Sentiment']==senti)]
                if df2['L3 Cluster ID'].nunique()<=min_l3_clusters:
                    df = df.append(df2,ignore_index=True)
                    continue

                l3_phrases = df2.drop_duplicates(['L3 Cluster ID'])['L3 Phrase Count'].mean()
                l3s = df2['L3 Cluster ID'].unique()
                x = []
                for l3 in l3s:
                    cnt = df2[df2['L3 Cluster ID']==l3]['L3 Phrase Count'].iloc[0]
                    if cnt<((percentage/100)*l3_phrases) or cnt<min_l3_phrases:
                        x.append(l3)
                if len(x)==0:
                    df = df.append(df2,ignore_index=True)
                    continue
                df = df.append(df2[~df2['L3 Cluster ID'].isin(x)])
                df2 = df2[df2['L3 Cluster ID'].isin(x)]
                df2['L3 Phrase Count'] = len(df2)
                df2['L3 Review Count'] = df2['Review ID'].nunique()
                df2['L3 Rating (Phrase)'] = df2['Review Rating'].mean()
                df2['L3 Rating (Review)'] = df2.drop_duplicates(['Review ID'])['Review Rating'].mean()    
                df2['L3 Cluster'] = 'Miscellaneous'
                df2['L3 Cluster ID'] = l2+'_misc_'+senti[0]
                if len(df2)==1:
                    for j in df2.index:
                        df2['L3 Cluster Phrase'][j] = df2['L3 Cluster Phrase'].iloc[0].strip().capitalize()                   
                    df = df.append(df2,ignore_index=True)
                    continue
                else:
                    df2['sim']=''
                    for i in df2.index:
                        sim=[]
                        for j in df2.index:
                            if i!=j:
                                sim.append(jellyfish.levenshtein_distance(df2['L4 Phrase'][i],df2['L4 Phrase'][j]))
                        df2['sim'][i] = sum(sim)/len(sim)
                    df2['L3 Cluster Phrase'] = df2[(df2['sim']==df2['sim'].min())]['L4 Phrase'].iloc[0].strip().capitalize()                   
                df = df.append(df2,ignore_index=True)
    return df.drop(['sim'],1)

In [8]:
def detailed_output(df,map_dict,client,take_sentiment=True,flag_rating=4,multi_product=False,compress=True,merging=False,
                    min_l3_clusters=3,min_l3_phrases=10,percentage=10,words_per_topic=5,n_neighbors=100,ng_range=(1,3),
                    words_to_remove=[],date=date.today().strftime('%Y%m%d'),version='v1',create_exl=False):
    ###################################Removing Outlier Clusters################################
    df3 = df
    #########################################sheet5##########################################
    sheet5 = pd.DataFrame(columns=['Sentiment','L1 Cluster ID','L1 Cluster','L1 Phrase Count','L1 Review Count',
                                   'L1 Rating (Phrase)','L1 Rating (Review)','L2 Cluster ID','L2 Cluster','L2 Phrase Count',
                                   'L2 Review Count','L2 Rating (Phrase)','L2 Rating (Review)','L3 Cluster ID','L3 Cluster',
                                   'L3 Cluster Phrase','L3 Phrase Count','L3 Review Count','L3 Rating (Review)',
                                   'L3 Rating (Phrase)','L4 ID','L4 Phrase','Review ID','Review Rating'])
    sheet5['Sentiment'] = df3.Sentiment
    sheet5['L2 Cluster'] = df3.Aspect
    sheet5['L4 Phrase'] = df3.Phrase
    sheet5['Review'] = df3.Review
    sheet5['Review ID'] = df3['Review ID']
    sheet5['Review Rating'] = df3.Rating
    sheet5['Date'] = pd.to_datetime(df3.Date)
    if multi_product:
        df3['Product'] = df['Review ID'].str.split('R',expand=True)[0] #.apply(lambda x:x[0]+x[2])#
        sheet5['Product'] = df3['Product']
#     sheet5 = sheet5.merge(daf.loc[:,['Review ID','Verified Purchase', 'Recommended',
#            'Incentivized', 'Upvote', 'Downvote', 'AgeRange', 'SkinConcerns']],on='Review ID')
    sheet5.fillna('',inplace=True)
    print("Clustering Started\n")
    
    #######################################Clustering###########################################
    print("BERTopic Clustering will take a while")
    nlp = spacy.load("en_core_web_sm")
    stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
     'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 
    'its','itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 
    'these','those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did',
    'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 
    'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',
    'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'only', 'own', 'same', 'so', 'than', 'too',
    'very', 's', 't','can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 
    'ain', 'aren', "aren't",'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
     "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', 
     "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't", 'since','first', 'even',
     'thisssss', 'could','really','always','bc','truly','literally','48','none','yet','actually','like','would','ever','issss',
    'although','much','10','do','super','c','f','definitely','completely','totally','ski','ive']+words_to_remove # 'not','no'
    def tokenization(text):
        return word_tokenize(text)
    def remove_stopwords(text):
        output= [i for i in text if i.lower() not in stopwords]
        return output

    df3.dropna(inplace=True)
#     v = CountVectorizer(ngram_range = ng_range,stop_words="english")
#     vd = v.fit_transform(list(df3.loc[:, "Phrase"].values))
    
    start_time = time.time()
    #Doing clustering for every aspect in every sentiment
    for senti in ['Positive','Negative']:
        for aspect in df3.Aspect.unique():
            if take_sentiment:
                df1 = df3[(df3.Aspect==aspect)&(df3.Sentiment==senti)]
                print('Sentiment:',senti,'Aspect:',aspect)
            else:
                df1 = df3[(df3.Aspect==aspect)]
                print('Aspect:',aspect)
            df1 = df1.reset_index()
            if len(df1)==0:
                print("There are no phrases for this aspect & sentiment.")
                continue

            scores = {}
            for x in words_per_topic:
              for y in n_neighbors:
                st = time.time()
                df1 = clustering(df1,y,x,ng_range,compress,nlp,tokenization,remove_stopwords,aspect)
                vectorizer = TfidfVectorizer()
                dv = vectorizer.fit_transform(df1['Phrase'])
                try:
                    sc = silhouette_score(dv,df1['Cluster'])
                    scores[sc] = [x,y]
                    et = time.time()
                    print("words_per_topic & n_neighbors:",[x,y])
                    print("silhouette_score:",sc)
                    print("Time Taken {:.2f} sec \n".format(et-st))
                except:
                    pass

            max_score = max(scores.keys())
            print("Max Score:",max_score) 
            print("words_per_topic & n_neighbors:",[scores[max_score][0],scores[max_score][1]])
            df1 = clustering(df1,scores[max_score][1],scores[max_score][0],ng_range,compress,nlp,tokenization,remove_stopwords,aspect)                   
            #This code will give us the most similar phrase inside each L3 Cluster
            df1 = most_similar_phrase(df1)

            #This code will generate the summary of L3 Clusters and store it in sheet5
            layer3 = summary_l3(df1,multi_product)
        
            if take_sentiment:
                s = sheet5[(sheet5['L2 Cluster']==aspect)&(sheet5.Sentiment==senti)]
            else:
                s = sheet5[(sheet5['L2 Cluster']==aspect)]
                
            for i in s.index:
                try:
                    if multi_product:
                        x = df1[(df1.Product==sheet5['Product'][i])&(df1.Phrase==sheet5['L4 Phrase'][i])&(df1['Review ID']==sheet5['Review ID'][i])&(df1['Sentiment']==sheet5['Sentiment'][i])]
                    else:
                        x = df1[(df1.Phrase==sheet5['L4 Phrase'][i])&(df1['Review ID']==sheet5['Review ID'][i])&(df1['Sentiment']==sheet5['Sentiment'][i])]
                    c = list(x['Cluster'])[0]
                    st = list(x['Sentiment'])[0]
                    sheet5['L3 Cluster'][i] = c
                    sheet5['L3 Cluster Phrase'][i] = list(x['L3 Cluster Phrase'])[0]
                    
                    if multi_product:
                        prod = list(x['Product'])[0]
                        y = layer3[(layer3.Product==prod)&(layer3.Cluster==c)&(layer3.Sentiment==st)]
                    else:
                        y = layer3[(layer3.Cluster==c)&(layer3.Sentiment==st)]
                    sheet5['L3 Phrase Count'][i] = list(y['L3 Phrase Count'])[0]
                    sheet5['L3 Review Count'][i] = list(y['L3 Review Count'])[0]
                    sheet5['L3 Rating (Phrase)'][i] = list(y['L3 Rating (Phrase)'])[0]
                    sheet5['L3 Rating (Review)'][i] = list(y['L3 Rating (Review)'])[0]
                except Exception as e:
#                     print(c,e)
                    pass
            
        if take_sentiment==False:
            break
    print("Clustering done after {:.2f} sec \n".format(time.time() - start_time))

    sheet5 = sheet5[sheet5['L3 Cluster'].astype(bool)] #To remove empty L3 clusters from sheet5
    
    
    ###############################Layer 2 & 1 Creation##############################################
#     return sheet5
    if multi_product:
        ps = sheet5.Product.unique()
        for product in ps:
            sheet5 = layer_creation(sheet5,'L2',multi_product,product)
    else:
        sheet5 = layer_creation(sheet5,'L2',multi_product)
    
    #This code is used to map L2 Clusters to L1 Clusters
    d = map_dict
    for l2 in sheet5['L2 Cluster'].unique():
        dfx = sheet5[sheet5['L2 Cluster']==l2]
        l1 = ''
        for key,values in dP1.items():
            if l2 in values:
                l1 = key
                break
        for i in dfx.index:
            sheet5['L1 Cluster'][i] = l1
        
    if multi_product:
        ps = sheet5.Product.unique()
        for product in ps:
            sheet5 = layer_creation(sheet5,'L1',multi_product,product)
    else:
        sheet5 = layer_creation(sheet5,'L1',multi_product)
    
    ###################################Id insertion & output excel formation################################
    for la in range(1,5):
        sheet5 = id_creation(sheet5,layer=la,client=client)
    
    if merging:
        sheet5 = cluster_merge(sheet5,multi_product,min_l3_clusters,min_l3_phrases,percentage)
    ###################################Output excel formation################################    
    #Creating different tabs in detailed output from sheet5 [layer 4]
    d1 = sheet5
    d2 = sheet5.iloc[:,:20]
    d3 = sheet5.iloc[:,:13]
    d4 = sheet5.iloc[:,:7]
    
    #Sorting values
    print("Sorting Started\n")
    if multi_product:
        d4['Product'] = d3['Product'] = d2['Product'] = d1['Product']
#         d1 = d1.sort_values(['Product','L1 Cluster','L2 Cluster','Sentiment','L3 Phrase Count','L3 Cluster'],ascending=False)
#         d2 = d2.sort_values(['Product','L1 Cluster','L2 Cluster','Sentiment','L3 Phrase Count','L3 Cluster'],ascending=False)
#         d3 = d3.sort_values(['Product','L1 Cluster','Sentiment','L2 Phrase Count','L2 Cluster'],ascending=False)
#         d4 = d4.sort_values(['Product','L1 Cluster','Sentiment'],ascending=False)
        #Removing Duplicates
        d2.drop_duplicates(['Product','Sentiment','L2 Cluster','L3 Cluster'],inplace=True)
        d3.drop_duplicates(['Product','Sentiment','L2 Cluster'],inplace=True)
        d4.drop_duplicates(['Product','Sentiment','L1 Cluster'],inplace=True)
    else:
#         d1 = d1.sort_values(['L1 Cluster','L2 Cluster','Sentiment','L3 Phrase Count','L3 Cluster'],ascending=False)
#         d2 = d2.sort_values(['L1 Cluster','L2 Cluster','Sentiment','L3 Phrase Count','L3 Cluster'],ascending=False)
#         d3 = d3.sort_values(['L1 Cluster','Sentiment','L2 Phrase Count','L2 Cluster'],ascending=False)
#         d4 = d4.sort_values(['L1 Cluster','Sentiment'],ascending=False)
        #Removing Duplicates
        d2.drop_duplicates(['Sentiment','L2 Cluster','L3 Cluster'],inplace=True)
        d3.drop_duplicates(['Sentiment','L2 Cluster'],inplace=True)
        d4.drop_duplicates(['Sentiment','L1 Cluster'],inplace=True)
    
    #Flagging [used to determine wishlist etc.]
    print("Flagging Started\n")
    for x in [d1,d2]:
        x['Flag']=''
        for i in x.index:
            if x['L3 Rating (Review)'][i]>=flag_rating:
                x['Flag'][i] = x['Sentiment'][i][0]+'+'
            else:
                x['Flag'][i] = x['Sentiment'][i][0]+'-'
        
    #Creating output excel file
    if create_exl:
        if multi_product:
            name = client+"_product"+"_detailed_"+date+version+".xlsx"
        else:
            name = client+"_category"+"_detailed_"+date+version+".xlsx"
        path = r"{fname}".format(fname=name)
        with pd.ExcelWriter(path) as engine:
            d1.to_excel(excel_writer=engine,sheet_name='L4',index=False)
            d2.to_excel(excel_writer=engine,sheet_name='L3',index=False)
            d3.to_excel(excel_writer=engine,sheet_name='L2',index=False)
            d4.to_excel(excel_writer=engine,sheet_name='L1',index=False)
        print("Excel File with 4 sheets is created.\n")
        print("File name:",name)
    dfs = [d1,d2,d3,d4]
    return dfs

In [9]:
#Hard coded dictionaries are created to club L2 aspects together to form L1 Clusters
dP1 = { 'Customer Demographics':['Value','Skin','Purchase','Motivation','Emotion','Demographic','Brand'],
         'Attributes':['Time','Partner','Odor','Ingredients','Design','Color','Application'],
        'Product Performance':['Finish','Coverage','Competitor']}

dP4 = {'Performance':['Allergy','Performance','Skin'],
         'Application':['Application','Texture'],
        'Emotional':['Brand','Competitor','Purchase','Sentiment'],
        'Product':['Demographic','Motivation','Odor','Packaging','Price']}

dP5 = {'Product Performance':['Outcome','Usage','Activity','Trust','Customerservice','Quality'],
      'Product Attributes':['Taste','Flavor','Value','Ingredient','Design','Odor','Smell'],
      'Customer Details':['Demographic','Brand','Competitor','Motivation']}

In [10]:
# l1 = detailed_output(df1,map_dict=dP1,client='foundation',take_sentiment=False,ng_range=(1,3),n_neighbors=15,
#                      words_per_topic = 1,words_to_remove=[],multi_product=False,compress=True,create_exl=False,version='v1')

# End Points

In [12]:
def top_n_reviews_l3(df,n=5,year_duration=2,review_length=None,phrases_per_review=None):
    #Filtering with year duration
    df1= df[df['Date']>=pd.to_datetime(date.today()-relativedelta(years=year_duration))]
    #Calculating the similarity of each phrase wrt the most common phrase
    
    if review_length!=None:
        df1['Review Length'] = df1['Review'].apply(lambda x:len(x))
        df1 = df1[df1['Review Length']>=review_length]
    if phrases_per_review!=None:
        df1['Phrases/Review'] = df1.groupby(['Review ID']).transform("count")['L4 ID']
        df1 = df1[df1['Phrases/Review']<=phrases_per_review]
        
    df1['sim']=''
    for id in df1['L3 Cluster ID'].unique():
        for senti in ['Positive','Negative']:
            for idx in df1[(df1['L3 Cluster ID']==id)&(df1['Sentiment']==senti)].index:
                df1['sim'][idx] = jellyfish.levenshtein_distance(df1['L3 Cluster Phrase'][idx],df1['L4 Phrase'][idx])
            
    #Creating the top_n_reviews dataframe
    df2 = pd.DataFrame(columns=df1.columns)
    for idx in df1['L3 Cluster ID'].unique():
        for senti in ['Positive','Negative']:
            s = len(df1[(df1['L3 Cluster ID']==idx)&(df1.Sentiment==senti)])
            #Appending the most recent phrase
#             try:
#                 df2 = df2.append(df1[(df1['L3 Cluster ID']==idx)&(df1.Sentiment==senti)].sort_values(['Date'],ascending=False).iloc[0])
#             except:
#                 pass
            #Appending rest n phrases in sorted order of similarity
            df3 = df1[(df1['L3 Cluster ID']==idx)&(df1.Sentiment==senti)].sort_values(['sim'])
            #&(df1['Review ID']!=df2.iloc[-1]['Review ID'])
            df3.drop_duplicates(['L3 Cluster ID','Review ID'],keep='first',inplace=True)
            df2 = df2.append(df3.iloc[:min(s,n)])
    return df2

In [13]:
def top_n_reviews_l2(df,client=None,n=5,version='v1',create_exl=False):
    df['sim']=''
    for id in df['L3 Cluster ID'].unique():
        for senti in ['Positive','Negative']:
            for idx in df[(df['L3 Cluster ID']==id)&(df['Sentiment']==senti)].index:
                df['sim'][idx] = -1*jellyfish.levenshtein_distance(df['L3 Cluster Phrase'][idx],df['L4 Phrase'][idx])
                
    df2 = pd.DataFrame(columns=df.columns)
    for idx in df['L2 Cluster ID'].unique():
        for senti in ['Positive','Negative']:
            df1 = df[(df['L2 Cluster ID']==idx)&(df['Sentiment']==senti)].sort_values(['L3 Review Count','sim'],ascending=False)
            if len(df1)==0:
                continue
            df2 = df2.append(df1.iloc[0])
            rid,cid = [],[]
            rid.append(df1.iloc[0]['Review ID'])
            cid.append(df1.iloc[0]['L3 Cluster ID'])
            cnt=n-1
            while(cnt):
                try:
                    df2 = df2.append(df1[(~df1['Review ID'].isin(rid))&(~df1['L3 Cluster ID'].isin(cid))].iloc[0])
                    rid.append(df2.iloc[-1]['Review ID'])
                    cid.append(df2.iloc[-1]['L3 Cluster ID'])
                    cnt-=1
                except:
                    break
    if create_exl:
        name = client+"_top_"+str(n)+"_reviews_l2_"+date.today().strftime('%Y%m%d')+version+".xlsx"
        df2.to_excel(name,index=False)
    return df2.drop(['sim'],1)

In [16]:
df = pd.read_excel('protein_product_detailed_20230318v1.xlsx')

In [15]:
top_n_reviews_l2(df).to_excel('protein_top_5_reviews_l2_category_20230318v1.xlsx',index=False)
top_n_reviews_l3(df).to_excel('protein_top_5_reviews_l3_category_20230318v1.xlsx',index=False)

In [18]:
daf = pd.DataFrame()
for p in df['Product'].unique():
    dfp = top_n_reviews_l2(df[df.Product==p])
    daf = pd.concat([daf, dfp], ignore_index=True)
# for i in daf[daf['L3 Cluster']!='Miscellaneous'].index:
#     daf['Flag2'][i]=1
daf.sort_values(['L2 Cluster ID','Product','Sentiment'],ascending=False).to_excel('protein_top_5_reviews_l2_product_20230318v1.xlsx',index=False)